In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# read in data and view 

data = pd.read_csv("/Users/victoriaguo/Desktop/DS 4002/project 1/original_data.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10261 non-null  object 
 1   asin            10261 non-null  object 
 2   reviewerName    10234 non-null  object 
 3   helpful         10261 non-null  object 
 4   reviewText      10254 non-null  object 
 5   overall         10261 non-null  float64
 6   summary         10261 non-null  object 
 7   unixReviewTime  10261 non-null  int64  
 8   reviewTime      10261 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 721.6+ KB
None


In [4]:
# check for NAs

data.isnull().sum()

# fill review text NA values as "missing" instead of empty
data['reviewText']=data['reviewText'].fillna('Missing')



In [5]:
# combine review text and summary columns
data['reviews']=data['reviewText']+data['summary']
data=data.drop(['reviewText', 'summary'], axis=1)

In [6]:
# create sentiment column
data['overall'].value_counts()

def f(row):    
    if row['overall'] == 3.0:
        val = 'Neutral'
    elif row['overall'] == 1.0 or row['overall'] == 2.0:
        val = 'Negative'
    elif row['overall'] == 4.0 or row['overall'] == 5.0:
        val = 'Positive'
    else:
        val = -1
    return val

data['sentiment'] = data.apply(f, axis=1)

In [7]:
# view our data with the added column 
data.head()

data['sentiment'].value_counts()


sentiment
Positive    9022
Neutral      772
Negative     467
Name: count, dtype: int64

In [8]:
# drop reviewierID, unixReviewTime, asin columns

data=data.drop(['asin'], axis=1)
data=data.drop(['reviewerID'], axis=1)
data=data.drop(['reviewerName'], axis=1)
data=data.drop(['unixReviewTime'], axis=1)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   helpful     10261 non-null  object 
 1   overall     10261 non-null  float64
 2   reviewTime  10261 non-null  object 
 3   reviews     10261 non-null  object 
 4   sentiment   10261 non-null  object 
dtypes: float64(1), object(4)
memory usage: 400.9+ KB


In [9]:
# change reviewTime column to date year format 
date_new = data["reviewTime"].str.split(",", n = 1, expand = True) 

data["date"]= date_new[0] 
data["year"]= date_new[1]

data["year"]= date_new[1] 

data=data.drop(['reviewTime'], axis=1)


In [10]:
# look at our cleaned dataset
data.head()

,helpful,overall,reviews,sentiment,date,year
0,"[0, 0]",5.0,"Not much to write about here, but it does exac...",Positive,02 28,2014
1,"[13, 14]",5.0,The product does exactly as it should and is q...,Positive,03 16,2013
2,"[1, 1]",5.0,The primary job of this device is to block the...,Positive,08 28,2013
3,"[0, 0]",5.0,Nice windscreen protects my MXL mic and preven...,Positive,02 14,2014
4,"[0, 0]",5.0,This pop filter is great. It looks and perform...,Positive,02 21,2014


In [11]:
# create new column with helpfulness rate of a review 
new1 = data["helpful"].str.split(",", n = 1, expand = True)
new2 = new1[0].str.split("[", n = 1, expand = True)
new3 = new1[1].str.split("]", n = 1, expand = True)

#Resetting the index
new2.reset_index(drop=True, inplace=True)
new3.reset_index(drop=True, inplace=True)

#Dropping empty columns due to splitting 
new2=new2.drop([0], axis=1)
new3=new3.drop([1], axis=1)

#Concatenating the splitted columns
helpful=pd.concat([new2, new3], axis=1)

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

#Applying the function
helpful= trim_all_columns(helpful)

#Converting into integer types
helpful[0]=helpful[0].astype(str).astype(int)
helpful[1]=helpful[1].astype(str).astype(int)

#Dividing the two columns, we have 0 in the second columns when dvided gives error, so I'm ignoring those errors
try:
  helpful['result'] = helpful[1]/helpful[0]
except ZeroDivisionError:
  helpful['result']=0

#Filling the NaN values(created due to dividing) with 0
helpful['result'] = helpful['result'].fillna(0)

#Rounding of the results to two decimal places
helpful['result']=helpful['result'].round(2) 

#Attaching the results to a new column of the main dataframe
data['helpful_rate']=helpful['result']

#dropping the helpful column from main dataframe
data=data.drop(['helpful'], axis=1)

In [12]:
# drop date column and keep year
data=data.drop(['date'], axis=1)


In [13]:
# look at dataset 
data.head()

,overall,reviews,sentiment,year,helpful_rate
0,5.0,"Not much to write about here, but it does exac...",Positive,2014,0.00
1,5.0,The product does exactly as it should and is q...,Positive,2013,0.93
2,5.0,The primary job of this device is to block the...,Positive,2013,1.00
3,5.0,Nice windscreen protects my MXL mic and preven...,Positive,2014,0.00
4,5.0,This pop filter is great. It looks and perform...,Positive,2014,0.00


In [14]:
# clean the review column 
def review_cleaning(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [15]:
# apply to review column in data
data['reviews']=data['reviews'].apply(lambda x:review_cleaning(x))

data.head()

,overall,reviews,sentiment,year,helpful_rate
0,5.0,not much to write about here but it does exact...,Positive,2014,0.00
1,5.0,the product does exactly as it should and is q...,Positive,2013,0.93
2,5.0,the primary job of this device is to block the...,Positive,2013,1.00
3,5.0,nice windscreen protects my mxl mic and preven...,Positive,2014,0.00
4,5.0,this pop filter is great it looks and performs...,Positive,2014,0.00


In [16]:
# drop the neutral reviews (only comparing positive and negative)

indexSentiment = data[(data['sentiment'] == 'Neutral')].index
data.drop(indexSentiment, inplace=True)

In [17]:
# export data to csv

data.to_csv("/Users/victoriaguo/Desktop/DS 4002/project 1/final_data.csv")
